In [146]:
import chess
import numpy as np
import pandas as pd
import numexpr as ne
import swifter

In [147]:
df = pd.read_csv("./random_evals.csv")
df.head(10)

,FEN,Evaluation
0,rnbqkb1r/pppppppp/B4n2/8/4P3/8/PPPP1PPP/RNBQK1...,-459
1,rnbqkb1r/pppppppp/5n2/1B6/4P3/8/PPPP1PPP/RNBQK...,-125
2,rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBN...,+198
3,rnbqkb1r/pppppppp/5n2/8/4P3/7N/PPPP1PPP/RNBQKB...,-155
4,rnbqkbnr/ppppp1pp/8/5p2/4P3/8/PPPP1PPP/RNBQKBN...,+209
5,rnbqkb1r/pppppppp/5n2/8/2B1P3/8/PPPP1PPP/RNBQK...,-101
6,rnbqkb1r/pppppppp/5n2/8/4P3/5N2/PPPP1PPP/RNBQK...,-105
7,rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPPNPPP/RNBQKB1...,-136
8,rnbqkb1r/pppppppp/5n2/7Q/4P3/8/PPPP1PPP/RNB1KB...,-1262
9,rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPPBPPP/RNBQK1N...,-117


In [148]:

    # PRE-PROCESSING


In [149]:
df['Evaluation'] = df['Evaluation'].astype(str)

In [150]:
df.loc[df['Evaluation'].str.contains("#-", na=False), 'Evaluation'] = -2000
df.loc[df['Evaluation'].str.contains("#+", na=False), 'Evaluation'] = 2000

In [151]:
df['Evaluation'] = df['Evaluation'].astype(int) / 100.

In [152]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))
def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1. - s)

def chessmoid(x):
    return sigmoid(0.9 * x)
def chessmoid_derivative(x):
    # chessmoid'(x) = sigmoid'(0.9 * x) * 0.9
    return sigmoid_derivative(0.9 * x) * 0.9

In [153]:
df['Prob'] = np.vectorize(chessmoid)(df['Evaluation'])

In [154]:
def fen_to_board(x):
    return chess.Board(fen=x)

In [155]:
df['Board'] = df['FEN'].swifter.apply(fen_to_board)

Pandas Apply:   0%|          | 0/1000273 [00:00<?, ?it/s]

In [156]:
def board_to_bools(board):
    arr = np.zeros(784 + 5, dtype=bool)
    idx = 0

    for col in range(0, 2):
        for pc in range(1, 6 + 1):
            BB = int(board.pieces(pc, col))
#             arr[(col) * 6 + (pc - 1)] += bin(BB).count("1")
            for i in range(0, 64):
                arr[idx] = (BB >> i) & 1
                idx += 1

    arr[idx] = (board.castling_rights & chess.BB_H1) != 0
    idx += 1
    arr[idx] = (board.castling_rights & chess.BB_A1) != 0
    idx += 1
    arr[idx] = (board.castling_rights & chess.BB_H8) != 0
    idx += 1
    arr[idx] = (board.castling_rights & chess.BB_A8) != 0
    idx += 1
    arr[idx] = board.turn
#     arr[5] = ((board.castling_rights & chess.BB_H1) != 0)\
#            + ((board.castling_rights & chess.BB_A1) != 0)
#     arr[11] = ((board.castling_rights & chess.BB_H8) != 0)\
#             + ((board.castling_rights & chess.BB_A8) != 0)
#     arr[12] = board.turn
    
    return arr

In [157]:
df['Bits'] = df['Board'].swifter.apply(board_to_bools)

Pandas Apply:   0%|          | 0/1000273 [00:00<?, ?it/s]

In [158]:
mask = np.random.rand(len(df)) < 0.8
train = df[mask]
test = df[~mask]

In [159]:

    # LINEAR REGRESSION


In [160]:
from sklearn.linear_model import LinearRegression
X = list(train['Bits'])
y = list(train['Prob'])
reg = LinearRegression().fit(X, y)

In [161]:
X_test = list(test['Bits'])
y_test = list(test['Prob'])
reg.score(X_test, y_test)

0.5972420069466631

In [162]:
import sys
def get_val(col, pc):
    pc_col = 6 * col + (pc - 1)
    idx_range = range(8, 56) if pc == chess.PAWN else range(0, 64)
    n = 48 if pc == chess.PAWN else 64
    v = sum(reg.coef_[64*pc_col:64*(pc_col+1)][idx_range]) / n
    return v

def norm_val(v):
    pv = get_val(chess.WHITE, chess.PAWN)
    return v / pv # 8 * (v - pv) / (qv - pv) + 1

def get_norm_val(col, pc):
    return norm_val(get_val(col, pc))

def print_sqs(col, pc):
    avg_val = get_norm_val(col, pc)
    print("AVG VAL", avg_val)
    pc_col = 6 * col + (pc - 1)
    arr = reg.coef_[ (64 * pc_col) : (64 * (pc_col + 1)) ]
    arr = [ (norm_val(x) - avg_val) for x in arr ]
    for i in range(0, 64):
        print(int(arr[i] * 10) / 10, end="\t")
        if i % 8 == 7:
            print()

In [163]:
print_sqs(chess.WHITE, chess.PAWN)

AVG VAL 1.0
3244586406.3	6086849979.7	-2670689962.9	4648553319.9	4330894784.3	-502560045.2	16892744411.4	-512620956.5	
-0.5	0.0	-0.3	-0.9	-0.4	-0.5	0.0	-0.4	
-0.3	0.0	-0.1	-0.5	-0.3	-0.3	0.0	-0.2	
-0.2	0.0	-0.1	-0.2	-0.2	-0.7	0.0	0.0	
-0.1	0.0	0.0	-0.3	0.0	-0.1	0.3	-0.3	
0.7	0.6	0.4	0.0	0.2	0.6	0.1	-0.2	
1.6	1.3	1.7	0.2	0.0	0.7	0.1	-0.8	
2254749952.7	-2917295797.8	4338594593.3	-7273738524.1	3695432816.6	-11483320496.4	-2809920138.7	10378421661.0	


In [164]:

    # NEURAL NETWORK


In [491]:
class Activation:
    def __init__(self, fn_name):
        if fn_name == 'relu':
            self.function   = lambda x : np.maximum(0, x)
            self.derivative = lambda x : x >= 0

        elif fn_name == 'relu_clamped':
            self.function   = lambda x : np.clip(x, 0, 1)
            self.derivative = lambda x : 0 <= x and x <= 1

        elif fn_name == 'identity':
            self.function   = lambda x : x
            self.derivative = lambda x : 1

        elif fn_name == 'sigmoid':
            self.function   = sigmoid
            self.derivative = sigmoid_derivative

        elif fn_name == 'chessmoid':
            self.function   = chessmoid
            self.derivative = chessmoid_derivative

        elif fn_name == 'exp_normalize':
            # v_i = e^v_i / (e^v_i + c)
            # dv_i = 
            self.function = lambda v : np.exp(v)
            self.derivative = lambda v : np.exp(v)
            return

        else:
            print("ERR: Unrecognized activation function")

        self.function   = np.vectorize(self.function)
        self.derivative = np.vectorize(self.derivative)

In [492]:
class Layer:
    def __init__(self, inputs, outputs, activation_fn_name):
        self.activation = Activation(activation_fn_name)
        self.inputs = inputs
        self.outputs = outputs
        
        max_weight = np.sqrt(2.0 / self.inputs)
        self.A = np.random.rand(outputs, inputs) * max_weight
        self.b = np.zeros((outputs, 1))

        self.backups = []

    def predict(self, x): # forward prop
        x = np.array(x).reshape(self.inputs, 1)
        y = np.matmul(self.A, x) + self.b # y = Ax + b
        z = self.activation.function(y)   # z = f(y)

        self.prev_x = x
        self.prev_y = y
        self.prev_z = z

        return z        

    def backprop(self, dC_dz): # backwards prop
        # y_i = sum_j (A_ij * x_j) + b_i
        # z_i = o(y_i)

        # dC_i/dy_i = dC_i/dz_i * dz_i/dy_i = dC_i/dz_i * o'(y_i)
        dC_dy = dC_dz * self.activation.derivative(self.prev_y)

        # dC_i/db_i = dC_i/dy_i * dy_i/db_i = dC_i/dy_i * 1
        dC_db = dC_dy

        # dC_i/dA_ij = dC_i/dy_i * dy_i/dA_ij = dC_i/dy_i * x_j
        dC_dA = np.empty((self.outputs, self.inputs))        
        for r in range(0, self.outputs):
            for c in range(0, self.inputs):
                dC_dA[r][c] = dC_dy[r][0] * self.prev_x[c][0]

        # dC_i/dx_i = dC_i/dy_i * dy_i/dx_i = dC_i/dy_i * A_ij
        dC_dx = np.zeros((self.inputs, 1))
        for c in range(0, self.inputs):
            for r in range(0, self.outputs):
                dC_dx[c][0] += dC_dy[r][0] * self.A[r][c]

        return dC_db, dC_dA, dC_dx

    def push_backup(self):
        self.backups.append((
            np.copy(self.A),
            np.copy(self.b)
        ))

    def pop_backup(self, nParams):
        (nA, nb) = self.backups.pop()
        self.A = nA
        self.b = nb

In [508]:
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers

    def predict(self, x):
        for layer in self.layers:
            x = layer.predict(x) # pipe result to next layer
        return x

    def cost(self, x, s):
        z = self.predict(x)
        delta = z - s
        delta = delta.T[0] # column vector -> array
        return np.dot(delta, delta)

    def backprop(self, x, s):
        z = self.predict(x) # gets result and sets prev_x/y/z

        delta = z - s
        delta_arr = delta.T[0] # column vector -> array
        C = np.dot(delta_arr, delta_arr)
        dC = 0.0 - C
        dC_dz = 2.0 * delta

        adjustments = []
        for layer in reversed(self.layers):
            dC_db, dC_dA, dC_dx = layer.backprop(dC_dz)

            db = np.divide(dC, dC_db, out = np.zeros_like(dC_db), where = dC_db != 0)
            dA = np.divide(dC, dC_dA, out = np.zeros_like(dC_dA), where = dC_dA != 0)
            adjustments.insert(0, [ db, dA ])

            dC_dz = dC_dx # pipe to next layer

        return adjustments

    def process_batch(self, df, in_field, out_field):
        n = len(df)

        in_col = df.columns.get_loc(in_field)
        out_col = df.columns.get_loc(out_field)
        
        results = df.swifter.progress_bar(False).apply(lambda r: self.backprop(r[in_col], r[out_col]), axis=1)
        results = np.array(results)

        total_adjs = results[0]
        for ri in range(1, n):
            res_adjs = results[ri]
            for i, layer_adjs in enumerate(res_adjs):
                for j, adj in enumerate(layer_adjs):
                    total_adjs[i][j] += adj / n

        return total_adjs

    def batch_average_cost(self, inputs, solutions):
        output = inputs.swifter.progress_bar(False).apply(self.predict) # predict batch
        return np.average(np.power(2.0, output - solutions)) # diff w/ sols

    def apply_adjustments(self, adjs, alpha):
        for i, adj in enumerate(adjs):
            layer = self.layers[i]
            layer.b += alpha * adj[0]
            layer.A += alpha * adj[1]

    def push_backup(self):
        for layer in self.layers:
            layer.push_backup()

    def pop_backup(self):
        for layer in enumerate(self.layers):
            layer.pop_backup()

    def train(
        self, 
        df, in_col, out_col,
        batch_size = 1_000, alpha = 1.0e-2,
        stopper = lambda err, i : i == 100
    ):
        i = 0

        while True:
            sample = df.sample(n = batch_size)
            ins = sample[in_col]
            outs = sample[out_col]
            adjs = self.process_batch(sample, in_col, out_col)

            err = self.batch_average_cost(ins, outs)
            print("perror:", err)

            self.apply_adjustments(adjs, alpha)
            err = self.batch_average_cost(ins, outs)
            print("ferror:", err)

            i += 1
            if stopper(err, i):
                break

In [527]:
nn = NeuralNetwork([
    Layer(784 + 5, 1, 'identity'),
#     Layer(10,  1, 'sigmoid')
])

In [1]:
nn.train(
    df = df, in_col = 'Bits', out_col = 'Evaluation',
    batch_size = 200,
    alpha = 0.0001,
    stopper = lambda err, i : err <= 0.13
)

NameError: name 'nn' is not defined

In [520]:
np.vectorize(nn.predict)(df['Bits'])

C:\Users\mehra\AppData\Local\Temp/ipykernel_7008/2106382036.py:2: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(-x))


array([0., 0., 0., ..., 0., 0., 0.])

In [514]:
nn.batch_average_cost(df['Bits'], df['Prob'])

array([[0.73101973]])

In [454]:
np.average(np.abs(df['Prob'] - df['Preds']))

array([[0.36596498]])

In [455]:
np.median(np.abs(df['Prob'] - df['Preds']))

array([[0.42258878]])

In [447]:
df['Preds'] = df['Bits'].swifter.apply(lambda x : nn.predict(x))

Pandas Apply:   0%|          | 0/1000273 [00:00<?, ?it/s]

In [400]:
df

,FEN,Evaluation,Prob,Board,Bits
0,rnbqkb1r/pppppppp/B4n2/8/4P3/8/PPPP1PPP/RNBQK1...,-4.59,0.015813,r n b q k b . r\np p p p p p p p\nB . . . . n ...,"[False, False, False, False, False, False, Fal..."
1,rnbqkb1r/pppppppp/5n2/1B6/4P3/8/PPPP1PPP/RNBQK...,-1.25,0.245085,r n b q k b . r\np p p p p p p p\n. . . . . n ...,"[False, False, False, False, False, False, Fal..."
2,rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBN...,1.98,0.855944,r n b q k b n r\np . p p p p p p\n. . . . . . ...,"[False, False, False, False, False, False, Fal..."
3,rnbqkb1r/pppppppp/5n2/8/4P3/7N/PPPP1PPP/RNBQKB...,-1.55,0.198611,r n b q k b . r\np p p p p p p p\n. . . . . n ...,"[False, False, False, False, False, False, Fal..."
4,rnbqkbnr/ppppp1pp/8/5p2/4P3/8/PPPP1PPP/RNBQKBN...,2.09,0.867726,r n b q k b n r\np p p p p . p p\n. . . . . . ...,"[False, False, False, False, False, False, Fal..."
...,...,...,...,...,...
1000268,r1b1k2r/p2p1pp1/4pb1p/qp6/NnP5/P7/4PPPP/1R1QKB...,-3.86,0.030061,r . b . k . . r\np . . p . p p .\n. . . . p b ...,"[False, False, False, False, False, False, Fal..."
1000269,r1b1k2r/p2p1pp1/1p2pb1p/q7/NnP5/P7/4PPPP/1R1QK...,2.55,0.908462,r . b . k . . r\np . . p . p p .\n. p . . p b ...,"[False, False, False, False, False, False, Fal..."
1000270,r1b1k2r/pp1p1pp1/q3pb1p/8/NnP5/P7/4PPPP/1R1QKB...,0.89,0.690188,r . b . k . . r\np p . p . p p .\nq . . . p b ...,"[False, False, False, False, False, False, Fal..."
1000271,r1b1k2r/pp1p1p2/4pbpp/q7/NnP5/P7/4PPPP/1R1QKBN...,3.33,0.952438,r . b . k . . r\np p . p . p . .\n. . . . p b ...,"[False, False, False, False, False, False, Fal..."


In [110]:
nn.push_backup()

AttributeError: 'Layer' object has no attribute 'push_backup'

In [1130]:

    # SCORE TO BEAT


In [1563]:
nn2 = NeuralNetwork([
    Layer(784, 1, chessmoid_activation),
])
nn2.layers[0].A = np.array([reg.coef_])
nn2.layers[0].b = np.array([[0.]])
ins = np.array(df['Bits'])
outs = np.array(df['Prob'])
nn2.batch_average_cost(ins, outs)

array([[0.1323163]])

In [1226]:

    # COMPARE W/ SKLEARN


In [1537]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 1), random_state=1)

In [ ]:
ins = [[float(x) for x in ls] for ls in ins]
outs = [float(x) for x in outs]

In [1556]:
X = [[0., 0.], [1., 1.]]
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X, y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [ ]:

    # MNIST


In [18]:
mdf = pd.read_csv('./mnist_test.csv')

In [19]:
mdf.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
def cols_to_img(row):
    return np.array(row)[1:(28*28 + 1)]

In [21]:
mdf['img'] = mdf.swifter.apply(cols_to_img, axis=1)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

In [22]:
def label_to_vec(label):
    arr = np.zeros(10, dtype=float)
    arr[int(label)] = 1.
    return arr

In [23]:
mdf['out'] = mdf['label'].apply(label_to_vec)

In [90]:
mdf['out'] = mdf['out'].apply(lambda x : np.array([x]).T)
mdf['img'] = mdf['img'].apply(lambda x : np.array([x]).T)

In [537]:
mnn = NeuralNetwork([
    Layer(784, 10, 'sigmoid')
])

In [538]:
mnn.backprop(mdf['img'][0], mdf['out'][0])[0][1].shape

(10, 784)

In [539]:
mnn.predict(mdf['img'][0])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [540]:
mnn.train(
    df = mdf, in_col = 'img', out_col = 'out',
    batch_size = 100,
    alpha = 0.0001
)

perror: [[1.94]
 [1.83]
 [1.89]
 [1.95]
 [1.86]
 [1.93]
 [1.87]
 [1.94]
 [1.88]
 [1.91]]
ferror: [[1.94]
 [1.83]
 [1.89]
 [1.95]
 [1.86]
 [1.93]
 [1.87]
 [1.94]
 [1.88]
 [1.91]]
perror: [[1.89]
 [1.88]
 [1.89]
 [1.89]
 [1.93]
 [1.87]
 [1.92]
 [1.89]
 [1.92]
 [1.92]]
ferror: [[1.89]
 [1.88]
 [1.89]
 [1.89]
 [1.93]
 [1.87]
 [1.92]
 [1.89]
 [1.92]
 [1.92]]
perror: [[1.9 ]
 [1.91]
 [1.96]
 [1.81]
 [1.89]
 [1.89]
 [1.92]
 [1.89]
 [1.95]
 [1.88]]
ferror: [[1.9 ]
 [1.91]
 [1.96]
 [1.81]
 [1.89]
 [1.89]
 [1.92]
 [1.89]
 [1.95]
 [1.88]]
perror: [[1.91]
 [1.84]
 [1.88]
 [1.92]
 [1.94]
 [1.9 ]
 [1.92]
 [1.88]
 [1.91]
 [1.9 ]]
ferror: [[1.91]
 [1.84]
 [1.88]
 [1.92]
 [1.94]
 [1.9 ]
 [1.92]
 [1.88]
 [1.91]
 [1.9 ]]
perror: [[1.94]
 [1.85]
 [1.88]
 [1.91]
 [1.91]
 [1.96]
 [1.92]
 [1.82]
 [1.84]
 [1.97]]
ferror: [[1.94]
 [1.85]
 [1.88]
 [1.91]
 [1.91]
 [1.96]
 [1.92]
 [1.82]
 [1.84]
 [1.97]]
perror: [[1.9 ]
 [1.95]
 [1.94]
 [1.86]
 [1.85]
 [1.95]
 [1.89]
 [1.94]
 [1.81]
 [1.91]]
ferror: [[1.9 ]
 [1.9

KeyboardInterrupt: 

In [ ]:
mnn.backprop()